<a href="https://colab.research.google.com/github/kahramankaya/Anlas-l-rEkonomiYoutube/blob/main/77.video(Two-Way%20ANOVA%20%C3%96rnek).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#devam 77.video(Two-Way ANOVA Örnek)
import numpy as np
import pandas as pd

np.random.seed(5)
list1=[]
list2=[]
for m in range(100):
  tohum_turlerı=np.random.choice(["a","b","c","d","e"])
  list1.append(tohum_turlerı)

for m in range(100):
  gubre_turlerı=np.random.choice(["x","y","z"])
  list2.append(gubre_turlerı)

verim=np.random.randint(1,1000,size=(100))
pd.DataFrame(verim,columns=["verim"])
data=pd.concat([pd.DataFrame(list1,columns=["tohum"]),pd.DataFrame(list2,columns=["gubre"]),pd.DataFrame(verim,columns=["verim"])],axis=1)
print(data)


   tohum gubre  verim
0      d     x    411
1      a     z    938
2      b     x    212
3      a     y    285
4      e     x    460
..   ...   ...    ...
95     e     y    436
96     d     z    726
97     e     x    114
98     d     y    785
99     b     y    865

[100 rows x 3 columns]


In [20]:
#iki yonlu anova yapısı arkada bır dogrusal model ile çalışıyor.
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats as ss
from statsmodels.formula.api import  ols
import matplotlib.pyplot as plt

#yapılan işlem şu: yalnızca tohum yalnız gubre ve daha sonra tohum gubrenın bırlıkte verıme etkısı . yazılan kod bu anlamda.
model="verim ~ C(tohum)+C(gubre):C(tohum)+C(gubre)"
test=ols(model,data=data).fit()


In [19]:
#resid modelde ki hataları hatalar degıskenıne atmayı saglar.
hatalar=test.resid

nor=stats.shapiro(hatalar)
#hatalar normal dagılmıs ılk adım tamamlandı. ikinci adım varyans homojenlıgı ıcın h0 kabul varsayarak yola devam. cunku o bıraz burada karıısık regresyonda ıslenecek.
print(nor)

ShapiroResult(statistic=0.9913063049316406, pvalue=0.769005537033081)


In [24]:
#model anlamlılıgına bakalım. eger ols regression results kısmında "Prob (F-statistic):             0.0588"  bu degere baktıgımız zaman eger 0.05 ten kucuk ıse o zaman model anlamlı olur.
print(test.summary())

                            OLS Regression Results                            
Dep. Variable:                  verim   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     1.759
Date:                Tue, 07 Mar 2023   Prob (F-statistic):             0.0588
Time:                        15:51:00   Log-Likelihood:                -697.01
No. Observations:                 100   AIC:                             1424.
Df Residuals:                      85   BIC:                             1463.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [28]:
#lm lineer model demek.
anova=sm.stats.anova_lm(test,type=2)
# PR(>F)  bu kısma bak. tohumun,gubrenın,her ıksının bırlıkte verım ustunden etkısı var mı burdn anlaşılır. 0.05 ten buyuk mu kucuk mu bak bu kadar.
#tohumun degerı alpha dan buyuk demek kı verım ustunde etksıı var. gubrenın kı yok. her ıkısı bırlıkte verım ustunde etkısı var.
print(anova)

                     df        sum_sq        mean_sq         F    PR(>F)
C(tohum)            4.0  4.637365e+05  115934.114702  1.485630  0.213731
C(gubre)            2.0  8.432801e+05  421640.065090  5.403077  0.006181
C(gubre):C(tohum)   8.0  6.142965e+05   76787.058876  0.983982  0.454285
Residual           85.0  6.633147e+06   78037.024118       NaN       NaN


In [34]:
#posthoc yapısı ile grupların içidekilerin aralarında ne gıbı bır fark var bunu gorecegız.
x=ss.multicomp.pairwise_tukeyhsd(endog=data["verim"],groups=data["tohum"])
#reject kısmı false ise her ıkı grup arasında farklılık yok demektır. true ıse fark vardır demektır.
print(x)

#verim ortalamalarına bakıp kımın kımden daha iyi verim veridğini gorebılırız.
print(data.groupby("tohum")["verim"].mean())
#print(data["tohum"]["verim"].mean())

  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj    lower    upper   reject
--------------------------------------------------------
     a      b  178.2071 0.2964  -75.3518 431.7661  False
     a      c   38.4286 0.9954 -241.5805 318.4376  False
     a      d   68.3155 0.9348 -174.1795 310.8104  False
     a      e  155.2857 0.4242   -95.162 405.7334  False
     b      c -139.7786 0.6454 -422.5739 143.0167  False
     b      d -109.8917 0.7258 -355.5986 135.8153  False
     b      e  -22.9214 0.9991 -276.4804 230.6376  False
     c      d   29.8869 0.9981 -243.0322  302.806  False
     c      e  116.8571 0.7736 -163.1519 396.8662  False
     d      e   86.9702 0.8559 -155.5247 329.4652  False
--------------------------------------------------------
tohum
a    377.142857
b    555.350000
c    415.571429
d    445.458333
e    532.428571
Name: verim, dtype: float64


In [37]:
y=ss.multicomp.pairwise_tukeyhsd(endog=data["verim"],groups=data["tohum"]+data["gubre"])
print(y)
print(data.groupby(["tohum","gubre"])["verim"].mean())

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
    ax     ay    -79.05    1.0  -635.3694  477.2694  False
    ax     az       1.2    1.0  -555.1194  557.5194  False
    ax     bx  -36.4667    1.0  -580.7691  507.8358  False
    ax     by     321.8 0.8827   -295.381   938.981  False
    ax     bz     281.7 0.9376  -309.2058  872.6058  False
    ax     cx     -88.3    1.0  -679.2058  502.6058  False
    ax     cy     186.2 0.9996  -468.4193  840.8193  False
    ax     cz    -23.05    1.0  -677.6693  631.5693  False
    ax     dx     -93.2    1.0  -627.6944  441.2944  False
    ax     dy     30.95    1.0  -525.3694  587.2694  False
    ax     dz     268.7 0.9567  -322.2058  859.6058  False
    ax     ex    -119.8    1.0  -936.2537  696.6537  False
    ax     ey     102.8    1.0  -431.6944  637.2944  False
    ax     ez  205.5333 0.9915  -338.7691  749.8358  Fal